In [ ]:
## setup analysis
execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
from analysis.lib.purification import purify_pq as ppq; reload(ppq)
from analysis.lib.purification import purify_BK as pbk; reload(pbk)
from analysis.lib.m2.ssro import pqsequence as pqs; reload(pqs)
from analysis.lib.pq import pq_tools;reload(pq_tools)
%matplotlib inline
from analysis.lib.fitting import fit, common


# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_early_data\20161208'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL2'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3 without magnet'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3 disting'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_noMag_InvalidData_LowRepump_OtherPulsePath_Ey_highstrain'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_noMag_InvalidData_v2'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_SIL3_Ex_noMag'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_lessPulses_moreReps_lowStrain_v2'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_lessPulses_moreReps_lowStrain_v2_LT3data'
# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_lessPulses_moreReps_highStrain_v3'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\LT3_lessPulses_moreReps_highStrain_v3_LT3data'

# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170210_Ex_highstrain_15_50reps'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170210_Ex_highstrain_15_50reps_LT3data'


# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170211_Ey_lowstrain_15_50reps'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170211_Ey_lowstrain_15_50reps_LT3data'

# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170211_Ey_lowstrain_7optpi_100reps'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170211_Ey_lowstrain_7optpi_100reps_LT3data'

# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170213_Ey_lowstrain_4optpi_200reps'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170213_Ey_lowstrain_4optpi_200reps_LT3data'

# folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170214_Ex_highstrain_15optpi_200reps'
# sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170214_Ex_highstrain_15optpi_200reps_LT3data'

folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170214_Ey_lowstrain_15optpi_200reps'
sec_folder = r'D:\measuring\analysis_data\telecom\HOM\SIL3\20170214_Ey_lowstrain_15optpi_200reps_LT3data'

folder = r'D:\measuring\analysis_data\telecom\HOM\LT4\20170225_LT4_Ex_2p5GHzStrSplit_15optpi_100reps20'


filename_contains = 'TPQI'
invalid_data = False
from analysis.lib.qTelecom import telecom_TPQI as TPQI; reload(TPQI)

In [ ]:
### define parameters

tpqi_dict = {
    'start_ch0' : 3233e3,#+197e3,
    'start_offset_ch1' : 2.4e3,
    'delta_attempts' : 100,
    'min_filter_attempts' : 1,
    'window_length' : 30e3,
    'pulse_sep' : 197e3,
    'number_of_pulses' : 14,
    'syncs_per_CR_check' : 100}

In [ ]:
### get coincidences
# execfile(r'D:\measuring\analysis\scripts\setup_analysis.py')
reload(TPQI)
reload(pq_tools)

dts, filtered_dts = TPQI.TPQI_analysis_w_invalid_data_marker(folder,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0']+tpqi_dict['start_offset_ch1'],
                                               tpqi_dict['window_length'],
                                               tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               use_invalid_data_marker = invalid_data,
                                               Base_Folder_secondary = sec_folder,
                                               filter_attempts = True,
                                               syncs_per_CR_check = tpqi_dict['syncs_per_CR_check'],
                                               delta_attempts = tpqi_dict['delta_attempts'],
                                               min_filter_attempts =tpqi_dict['min_filter_attempts'],                  
                                               contains = filename_contains,
                                               force_coincidence_evaluation = False,
                                               Verbose = False )
len(dts),len(filtered_dts)

In [ ]:
### plot result

lim = 300*2

hist_binsize = 2

bins = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,hist_binsize)
y,x=np.histogram(filtered_dts, bins=bins)
bin_centres = x[:-1] + (x[1] - x[0])/2
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(bin_centres,y)
ax.set_xlabel('Difference time (ns)')
ax.set_ylabel('Coincidences in bin')
# ax.set_range
ax.set_title('TPQI')
ax.set_xlim([-lim,lim])
# ax.set_ylim([0,30])

# envelope = lambda t: (75/50) * 5* np.exp(-abs(t)**2/20**2.)*(1-np.cos(2*np.pi*0.00.5*(pulse_sep/1e3)*t))/2
# #envelope = lambda t: (75/50) * 5* np.exp(-abs(t)**2/20**2.)

# xp = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,1)
# env = envelope(xp)
# ax.plot(xp,env)

plt.show()
plt.close('all')


# ### plot result

# bins = np.arange(min(filtered_dts_disting)-100.5,max(filtered_dts_disting)+100.5,hist_binsize)
# y,x=np.histogram(filtered_dts_disting, bins=bins)
# bin_centres = x[:-1] + (x[1] - x[0])/2
# fig = plt.figure()
# ax = fig.add_subplot(111)
# ax.plot(bin_centres,y)
# ax.set_xlabel('Difference time (ns)')
# ax.set_ylabel('Coincidences in bin')
# # ax.set_range
# ax.set_title('TPQI')
# ax.set_xlim([-lim,lim])
# # ax.set_ylim([0,30])

# envelope_dist = lambda t: 10* np.exp(-abs(t)**2/20**2.)
# xp = np.arange(min(filtered_dts)-100.5,max(filtered_dts)+100.5,1)
# env = envelope_dist(xp)
# ax.plot(xp,env)
# plt.show()
# plt.close('all')

In [ ]:
def fit_tpqi(g_a,n_pulses):
    """
    fitfunction for TPQI
        y(x) = a * (n_pulses - x)/n_pulses

    I.g.:
        g_a : amplitude
    """
    fitfunc_str = 'a'

    a = fit.Parameter(g_a, 'a')
    n_pulses = fit.Parameter(n_pulses, 'n_pulses')
    p0 = [a, n_pulses]

    def fitfunc(x):
        return a() * (n_pulses() - x)/n_pulses()

    return p0, fitfunc, fitfunc_str


number_of_pulses = tpqi_dict['number_of_pulses']
pulse_sep = tpqi_dict['pulse_sep']

counts = np.zeros(number_of_pulses)
diff =  range(1,number_of_pulses+1)
for z in diff:
    c_left = np.sum((filtered_dts<(0.45*(pulse_sep/1e3)-z*pulse_sep/1e3)) & (filtered_dts>(-0.45*(pulse_sep/1e3)-z*pulse_sep/1e3)))
    c_right = np.sum((filtered_dts<(0.45*(pulse_sep/1e3)+z*pulse_sep/1e3)) & (filtered_dts>(-0.45*(pulse_sep/1e3)+z*pulse_sep/1e3)))
    counts[z-1] = (c_left + c_right)/2
    
reload(fit)
fitted = fit.fit1d(diff[1:],counts[1:],fit_tpqi,300,10,ret=True)

plt.errorbar(diff, counts,yerr=np.sqrt(counts), fmt='o')
plt.plot(diff,fitted['fitfunc'](diff))
plt.xlabel('Pulse difference')
plt.ylabel('Coincidences')
delta = plt.xlim()[1] - plt.xlim()[0]
plt.xlim([plt.xlim()[0]-0.1*delta,plt.xlim()[1]+0.1*delta])
fitted_counts_in_next_pulse = fitted['params'][0]
print 'Fitted coincidences with next pulse', fitted_counts_in_next_pulse

In [ ]:
counts_in_next_pulse = (np.sum((filtered_dts<(-pulse_sep/1e3+0.5*(pulse_sep/1e3)))& (filtered_dts>(-pulse_sep/1e3-0.5*(pulse_sep/1e3))))+np.sum((filtered_dts>pulse_sep/1e3-0.5*(pulse_sep/1e3)) & (filtered_dts<pulse_sep/1e3+0.5*(pulse_sep/1e3))))/2
counts_in_same_pulse = np.sum((filtered_dts<0.5*(pulse_sep/1e3)) & (filtered_dts>-0.5*(pulse_sep/1e3)))
print counts_in_next_pulse

print 'counts left',np.sum((filtered_dts<-pulse_sep/1e3+0.5*(pulse_sep/1e3)) & (filtered_dts>-pulse_sep/1e3-0.5*(pulse_sep/1e3)))
print 'counts right',np.sum((filtered_dts>pulse_sep/1e3-0.5*(pulse_sep/1e3)) & (filtered_dts<pulse_sep/1e3+0.5*(pulse_sep/1e3)))
print 'averaged coincidences with following pulses',counts_in_next_pulse 
print 'coincidences in same pulse',counts_in_same_pulse

print 'fitted counts ', fitted_counts_in_next_pulse

Visibility = 2/3.

print 'distiungishable expected vis', Visibility
print 'measured Visibility', counts_in_same_pulse/float(counts_in_next_pulse)
print 'fitted Visibility', counts_in_same_pulse/float(fitted_counts_in_next_pulse)
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility
print 'normalized TPQI visibility', 1-(counts_in_same_pulse/float(fitted_counts_in_next_pulse))/Visibility


In [ ]:
def sweep_tpqi_param(param_name,sweep_pts,contains = filename_contains):
    vis = []
    vis_u = []
    counts_in_next_pulse_sweep = []
    
    initialVal = tpqi_dict[param_name]
    for p in sweep_pts:
        tpqi_dict[param_name] = p
        dts, filtered_dts = TPQI.TPQI_analysis_w_invalid_data_marker(folder,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0']+tpqi_dict['start_offset_ch1'],
                                               tpqi_dict['window_length'],
                                               tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               use_invalid_data_marker = invalid_data,
                                               Base_Folder_secondary = sec_folder,
                                               filter_attempts = True,
                                               syncs_per_CR_check = tpqi_dict['syncs_per_CR_check'],
                                               delta_attempts = tpqi_dict['delta_attempts'],
                                               min_filter_attempts =tpqi_dict['min_filter_attempts'],                  
                                               contains = filename_contains,
                                               force_coincidence_evaluation = False,
                                               Verbose = False )
        ## calc visibility
        counts_in_next_pulse = float((np.sum((filtered_dts<-pulse_sep-0.5*(pulse_sep/1e3)) & \
                                       (filtered_dts>-pulse_sep+0.5*(pulse_sep/1e3)))+np.sum((filtered_dts>pulse_sep-0.5*(pulse_sep/1e3)) & \
                                       (filtered_dts<pulse_sep+0.5*(pulse_sep/1e3))))/2.)
        counts_in_same_pulse = float(np.sum((filtered_dts<0.5*(pulse_sep/1e3)) & (filtered_dts>-0.5*(pulse_sep/1e3))))
        
        counts = np.zeros(number_of_pulses)
        diff =  range(1,number_of_pulses+1)
        for z in diff:
            c_left = np.sum((filtered_dts<(0.5*(pulse_sep/1e3)-z*pulse_sep/1e3)) & (filtered_dts>(-0.5*(pulse_sep/1e3)-z*pulse_sep/1e3)))
            c_right = np.sum((filtered_dts<(0.5*(pulse_sep/1e3)+z*pulse_sep/1e3)) & (filtered_dts>(-0.5*(pulse_sep/1e3)+z*pulse_sep/1e3)))
            counts[z-1] = (c_left + c_right)/2
        fitted = fit.fit1d(diff,counts,fit_tpqi,300,10,ret=True,do_print = False)
        counts_in_next_pulse = fitted['params'][0]
#         print counts_in_next_pulse

        Visibility = 2/3.
    
        counts_in_next_pulse_sweep.append(counts_in_next_pulse)
        
        if counts_in_next_pulse != 0:
            vis.append(1-(counts_in_same_pulse/float(counts_in_next_pulse))/Visibility)
            
            if counts_in_same_pulse == 0:
                counts_in_same_pulse_for_error = 1
            else:
                counts_in_same_pulse_for_error = counts_in_same_pulse
            vis_u.append(np.sqrt((counts_in_same_pulse_for_error**2)/counts_in_next_pulse**3+ \
                                 (counts_in_same_pulse_for_error)/counts_in_next_pulse**2)/(Visibility))
#             print counts_in_same_pulse, counts_in_next_pulse,vis_u[-1]
        else:
            vis.append(0)
            vis_u.append(0)
    
    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot(111)
    plt.errorbar(sweep_pts,vis,vis_u)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Visibility')
    deltaX = 0.1*(plt.xlim()[1] - plt.xlim()[0])
    x_limits = [plt.xlim()[0]-deltaX,plt.xlim()[1]+deltaX]
    plt.xlim(x_limits)
    if plt.ylim()[1] > 1.05:
        plt.ylim([plt.ylim()[0],1.05])
#     fig = plt.figure()
#     ax = plt.subplot(111)
#     plt.plot(sweep_pts,counts_in_next_pulse_sweep)
#     ax.set_xlabel(param_name)
#     ax.set_ylabel('Counts in next pulse')
#     plt.xlim(x_limits)
    plt.show()
    plt.close('all')
    
    tpqi_dict[param_name] = initialVal

In [ ]:
##### sweep tpqi parameters

sweep_tpqi_param('start_ch0',np.linspace(tpqi_dict['start_ch0']-4e3,tpqi_dict['start_ch0']+6e3,12),contains = filename_contains)
sweep_tpqi_param('window_length',np.linspace(10e3,60e3,15),contains = filename_contains)

In [ ]:
##### sweep tpqi parameters

sweep_tpqi_param('delta_attempts',np.linspace(10,200,10),contains = filename_contains)

delta_attempts = tpqi_dict['delta_attempts']
tpqi_dict['delta_attempts'] = 40
sweep_tpqi_param('min_filter_attempts',np.arange(1,200,40),contains = filename_contains)
tpqi_dict['delta_attempts'] = delta_attempts

In [ ]:
def sweep_tpqi_param_tail(param_name,sweep_pts,contains = filename_contains):
    ph0s = []
    ph1s = []
    initialVal = tpqi_dict[param_name]
    for p in sweep_pts:
        tpqi_dict[param_name] = p
        filtered_ph0, filtered_ph1 = TPQI.TPQI_analysis_tail(folder,
                                               tpqi_dict['start_ch0'],tpqi_dict['start_ch0']+tpqi_dict['start_offset_ch1'],
                                               tpqi_dict['window_length'],
                                               tpqi_dict['pulse_sep'],
                                               tpqi_dict['number_of_pulses'],
                                               filter_attempts = True,
                                               syncs_per_CR_check = tpqi_dict['syncs_per_CR_check'],
                                               delta_attempts = tpqi_dict['delta_attempts'],
                                               min_filter_attempts =tpqi_dict['min_filter_attempts'],                  
                                               contains = filename_contains,
                                               force_coincidence_evaluation = False,
                                               Verbose = False )

        ph0s.append(filtered_ph0)
        ph1s.append(filtered_ph1)

    ### do some plotting
    fig = plt.figure()
    ax = plt.subplot(111)
    plt.plot(sweep_pts,ph0s)
    plt.plot(sweep_pts,ph1s)
    ax.set_xlabel(param_name)
    ax.set_ylabel('Tail counts')
    plt.show()
    plt.close('all')
    
    tpqi_dict[param_name] = initialVal

In [ ]:
##### sweep tpqi parameters

sweep_tpqi_param_tail('delta_attempts',np.linspace(20,200,10),contains = filename_contains)

delta_attempts = tpqi_dict['delta_attempts']
tpqi_dict['delta_attempts'] = 40
sweep_tpqi_param_tail('min_filter_attempts',np.arange(20,200,20),contains = filename_contains)
tpqi_dict['delta_attempts'] = delta_attempts

In [ ]:
reload(ppq)
def check_tail(st_start,st_len, start_offset_ch1):

    tails_w1_1 = []
    tails_w1_2 = []

    filepaths_primary = tb.get_all_msmt_filepaths(folder,pattern = 'TPQI')

    # print filepaths
    for i,fp in enumerate(filepaths_primary):

        fp = os.path.split(fp)[0]

        a_lt4 = pqs.PQSequenceAnalysis(fp)

        ### analysis for channel 0  && window 1
        w1_ch0 = get_total_number_of_clicks_in_window(a_lt4,0,st_start,st_len)
        w1_ch1 = get_total_number_of_clicks_in_window(a_lt4,1,st_start,st_len,start_offset_ch1)
        last_sync = a_lt4.pqf['/PQ_sync_number-1'][-1]
        tail_w1_1 = (1e4)*float(w1_ch0)/last_sync

        tail_w1_2 = (1e4)*float(w1_ch1)/last_sync

        # print 'tail in w1 / w2 (1e-4)    ', tail_w1, ' / ', tail_w2
        tails_w1_1.append(tail_w1_1);
        tails_w1_2.append(tail_w1_2);

        if tail_w1_1 < 0.5:
            print fp
            
        a_lt4.finish()
    f = plt.figure()
    plot_data(range(len(tails_w1_1)),tails_w1_1,label = 'w1')
    plot_data(range(len(tails_w1_2)),tails_w1_2,label = 'w1')
    
    plt.legend(bbox_to_anchor=(1.05, 1), loc=2, borderaxespad=0.)
    # return np.array(tails_w1),np.array(tails_w2) 

def plot_data(x,y,**kw):
    label = kw.pop('label',None)
    y_u = kw.pop('y_u',None)
    if y_u != None:
        plt.errorbar(x,y,y_u,fmt = 'x',label = label,**kw)
    else: plt.plot(x,y,'x',label = label)
            
def get_total_number_of_clicks_in_window(a,channel,st_start,st_len,offset = 0):
            
        is_ph = pq_tools.get_photons(a.pqf)[channel]
        clicks = a.pqf['/PQ_sync_time-1'].value[np.where(is_ph)] + offset
        bins = np.arange(st_start-.5,st_start+st_len,1e3)
        y,x=np.histogram(clicks, bins=bins)
        x=x[:-1]
        # print 'Total clicks:', np.sum(y)

        return np.sum(y)
    
check_tail(tpqi_dict['start_ch0'],tpqi_dict['window_length'],tpqi_dict['start_offset_ch1'])

In [ ]:
reload(pq_tools)
# def get_total_number_of_clicks_in_window(a,channel,st_start,st_len,offset = 0):
            
#         is_ph = pq_tools.get_photons(a.pqf)[channel]
#         clicks = a.pqf['/PQ_sync_time-1'].value[np.where(is_ph)] + offset
#         bins = np.arange(st_start-.5,st_start+st_len,1e3)
#         y,x=np.histogram(clicks, bins=bins)
#         x=x[:-1]
#         # print 'Total clicks:', np.sum(y)

#         return np.sum(y)
    
# get_total_number_of_clicks_in_window(a,ch0,start_offset_ch1,window_length,)

# pq_tools.get_photon_hists_from_folder(folder,index= 1,pq_device = '',start= 0,  length =6e6, hist_binsize = 1000, save = False, log=False)


# filepaths = tb.get_all_msmt_filepaths(folder)
# for i,f in enumerate(filepaths):
#     if i == 0:
#         pqf = pq_tools.pqf_from_fp(f, rights = 'r+')
#         pq_tools.plot_photon_hist(pqf,pq_device = '',start= 0,  length =6e6, hist_binsize = 1000, save = False, log=False, index = 1)

pq_tools.plot_photon_hist_folder(folder,pq_device = '',start= 5.943e6+1*0.197e6,  
                                 length = 0*0.2e6+0.10e6, hist_binsize = 20, save = False, index = 1)


In [ ]:
reload(pq_tools)
i=0
long_path = 5.5e6
short_path = 2e6
is_ph0, is_ph1 = pq_tools.get_photon_hists_from_folder(folder,pq_device = '',start= 2446.5e3-10e3+15*197e3,  length =50e3, hist_binsize = 300, save = False, log=False, index = 1)
for i in range(len(is_ph0[0])):
    s=np.sum(is_ph0[0])
print s
for i in range(len(is_ph1[0])):
    s1=np.sum(is_ph1[0])
print s1